Relies on calling notebook to populate some parameters... hmmm... not great. But I don't want a long spaghetti jupyter notebook either. I just need to finish my thesis I can worry about writing good code at my future job :/

used with [domain_adaptation_1](./domain_adaptation_1.ipynb)

## Model-specific parameters

In [ ]:
model_file = "/tigress/kendrab/analysis-notebooks/model_outs/14-06-24/F143444_modelfile.tar"

model_name = "F"
# hyperparameters
padding_length = 39  # amount of data on each side of each segment for additional info
stride = 11  # size (and therefore spacing) of each segment
input_length = stride + 2*padding_length
kernel_size = 3
pool_size = 5
out_channels = 36  # like 'filters' in keras
thinning_factor = 0.5445523804834298
learning_rate = 0.0025266393119259896
batch_size = 11  # idk what this should be for best performance 
try:
    override_batch_size
except NameError:
    override_batch_size = None
if override_batch_size:  # doing this instead of refactoring and 
    # potentially majorly fucking things up. very dumb and bad
    batch_size = n
num_conv = 1
dropout_fraction = 0.11224778904622225
kp_limit = 9

hyperparams = {'learning_rate':learning_rate, 'out_channels':out_channels, 'kernel_size':kernel_size, 'pool_size':pool_size,
              'input_length':input_length, 'stride':stride, 'thinning_factor':thinning_factor}

## Model class definition

In [ ]:
def repeat_layers_n_times(layer_list, n):  # this instead of something simpler to be absolutely sure the layers are different objects and not repeating the same one
    new_layer_list = []
    for i in range(n):
        for layer in layer_list:
            new_layer_list.append(copy.deepcopy(layer))
    return new_layer_list

In [ ]:
#TODO feed hyperparameters into __init__
class ModelF(nn.Module):
    """ 1D CNN Model """
    def __init__(self):  # TODO change optuna params from globals to inputs to init
        super().__init__()
        # define these all separately because they will get different weights
        # consider smooshing these together into one convolution with in_channels=6. Idk if a good idea
        self.bx_layers = nn.Sequential(*repeat_layers_n_times([nn.LazyConv1d(out_channels, kernel_size, padding='valid'),
                                       nn.ReLU(), nn.MaxPool1d(pool_size), nn.Dropout(p=dropout_fraction)], num_conv))
        self.by_layers = nn.Sequential(*repeat_layers_n_times([nn.LazyConv1d(out_channels, kernel_size, padding='valid'),
                                       nn.ReLU(), nn.MaxPool1d(pool_size), nn.Dropout(p=dropout_fraction)], num_conv))
        self.bz_layers = nn.Sequential(*repeat_layers_n_times([nn.LazyConv1d(out_channels, kernel_size, padding='valid'),
                                       nn.ReLU(), nn.MaxPool1d(pool_size), nn.Dropout(p=dropout_fraction)], num_conv))
        self.ex_layers = nn.Sequential(*repeat_layers_n_times([nn.LazyConv1d(out_channels, kernel_size, padding='valid'),
                                       nn.ReLU(), nn.MaxPool1d(pool_size), nn.Dropout(p=dropout_fraction)], num_conv))
        self.ey_layers = nn.Sequential(*repeat_layers_n_times([nn.LazyConv1d(out_channels, kernel_size, padding='valid'),
                                       nn.ReLU(), nn.MaxPool1d(pool_size), nn.Dropout(p=dropout_fraction)], num_conv))
        self.ez_layers = nn.Sequential(*repeat_layers_n_times([nn.LazyConv1d(out_channels, kernel_size, padding='valid'),
                                       nn.ReLU(), nn.MaxPool1d(pool_size), nn.Dropout(p=dropout_fraction)], num_conv))
        self.jy_layers = nn.Sequential(*repeat_layers_n_times([nn.LazyConv1d(out_channels, kernel_size, padding='valid'),
                                       nn.ReLU(), nn.MaxPool1d(pool_size), nn.Dropout(p=dropout_fraction)], num_conv))

        self.post_merge_layers = nn.Sequential(nn.Conv1d(out_channels, out_channels*2, kernel_size,
                                                                     padding='valid'), nn.ReLU(),
                                                           nn.MaxPool1d(pool_size), nn.Dropout(p=dropout_fraction),
                                                nn.Flatten(),
                                                nn.LazyLinear(stride))


    def forward(self, bx, by, bz, ex, ey, ez, jy):
        bx_proc = self.bx_layers(bx)
        by_proc = self.by_layers(by)
        bz_proc = self.bz_layers(bz)
        ex_proc = self.ex_layers(ex)
        ey_proc = self.ey_layers(ey)
        ez_proc = self.ez_layers(ez)
        jy_proc = self.jy_layers(jy)
        combined = (bx_proc + by_proc + bz_proc + ex_proc + ey_proc + ez_proc + jy_proc)/6.
        logits = self.post_merge_layers(combined)

        return logits


## load the model into 'model'

In [9]:
def import_model_state(model):
    optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
    checkpoint = torch.load(model_file, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss_fn = checkpoint['loss_fn']

    model.eval()  # set to correct mode to get the correct results

In [10]:
if __name__ == "__main__":
    # this will be called if running the notebook, even from another notebook
    model = ModelF().to(device=device, dtype=torch.double)
    import_model_state(model)

['/home/kendrab/.conda/envs/torch-env/lib/python3.10/site-packages/ipykernel_launcher.py', '-f', '/home/kendrab/.local/share/jupyter/runtime/kernel-6e46495a-f050-476f-96cd-5ec60679bacf.json']
/home/kendrab/.conda/envs/torch-env/lib/python3.10/site-packages/ipykernel_launcher.py
success
